In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'digit-recognizer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3004%2F861823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240812%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240812T182821Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D26eb6d6875d44db9a49d48c6e0d760c40a2b2842c70f2724891dc9b5d79aebd5ed6032f9b4bb18b89a0c435e22d1de457701c2835d97fa709de6e6d73e37843939216550d3f52be77d7d3da4230050fd30cb80d1e04b4a0044ee382eb7fb59ca35221a3deffea03ed0b302773da566964a0a4616e382e7158cc0db38f123a0c6f16a61779298259bc4ee5c249071f14c02af6f1cee7152a0dc06ce144326277c5095f18dfcbbe75c99311dc85f6b560b247f1caa2e6a6f654a023da423944374b11fb882e1d89f4199cb7e4f30c52372522856a9a563bd3acb95eb94043d99fe32a09fef8ed6c1bd79395b311e0615f84729349b8c636ac71dac4080bef77cf9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

d = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
d = np.array(d)
m, n = d.shape
np.random.shuffle(d)

d_dev = d_[0:1000].T
Y_dev = d_dev[0]
X_dev = d_dev[1:n].T
X_dev = X_dev / 255.0

d_train = d_[1000:m].T
y_train = d_train[0]
x_train = d_train[1:n].T
x_train = x_train / 255.0
_, m_train = x_train.shape

class Linear:
    """
    Parameters:
    - input_dim (int): No of input features
    - output_dim (int): No of output features
    """
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.randn(input_dim, output_dim) * 0.01
        self.bias = np.zeros((1, output_dim))

    def forward(self, X):
        """
        Parameters:
        - X (ndarray): Input data

        Returns:
        - ndarray: Output data
        """
        self.input = X
        return np.dot(X, self.weights) + self.bias

    def backward(self, g_output):
        """
        Parameters:
        - g_output (ndarray): Grad from next layer

        Returns:
        - ndarray: Grad with respect to input
        """
        g_input = np.dot(g_output, self.weights.T)
        self.g_weights = np.dot(self.input.T, g_output)
        self.g_bias = np.sum(g_output, axis=0, keepdims=True)
        return g_input

class ReLU:
    def forward(self, X):
        """
        Parameters:
        - X (ndarray): Input data

        Returns:
        - ndarray: Output after ReLU activation
        """
        self.input = X
        return np.maximum(0, X)

    def backward(self, g_output):
        """
        Parameters:
        - g_output (ndarray): Grad from next layer

        Returns:
        - ndarray: Grad with respect to input
        """
        g_input = g_output * (self.input > 0)
        return g_input

class Softmax:
    def forward(self, X):
        """
        Parameters:
        - X (ndarray): Input data

        Returns:
        - ndarray: Output
        """
        exp_values = np.exp(X - np.max(X, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        return self.output

    def backward(self, g_output):
        """
        Parameters:
        - g_output (ndarray): Grad  from next layer

        Returns:
        - ndarray: Grad  with respect to input
        """
        return g_output

class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        """
        Parameters:
        - y_pred (ndarray): Predicted probabilities
        - y_true (ndarray): True labels

        Returns:
        - float: Loss value
        """
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-12, 1. - 1e-12)
        correct_confidences = y_pred_clipped[range(samples), y_true]
        return -np.mean(np.log(correct_confidences))

    def backward(self, y_pred, y_true):
        """
        Parameters:
        - y_pred (ndarray): Predicted probabilities
        - y_true (ndarray): True labels

        Returns:
        - ndarray: Gradient with respect to input
        """
        samples = len(y_pred)
        g_ = y_pred
        g_[range(samples), y_true] -= 1
        return g_ / samples

class SGD:
    """
    Parameters:
    - learning_rate (float)
    """
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def step(self, layers):
        """
        Parameters:
        - layers (list): List of model layers to update
        """
        for layer in layers:
            if hasattr(layer, 'weights'):
                layer.weights -= self.learning_rate * layer.g_weights
                layer.bias -= self.learning_rate * layer.g_bias

class Model:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.optimizer = None

    def add_layer(self, layer):
        """
        Parameters:
        - layer
        """
        self.layers.append(layer)

    def compile(self, loss, optimizer):
        """
        Parameters:
        - loss: function
        - optimizer: Optimizer instance
        """
        self.loss = loss
        self.optimizer = optimizer

    def forward(self, X):
        """
        Parameters:
        - X (ndarray): Input data

        Returns:
        - ndarray: final output
        """
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, g_output):
        """
        Parameters:
        - g_output (ndarray): Gradient of loss with respect to output
        """
        for layer in reversed(self.layers):
            g_output = layer.backward(g_output)

    def train(self, X, y, epochs):
        """
        Parameters:
        - X (ndarray): Training data
        - y (ndarray): True labels
        - epochs (int): Number of epochs to train
        """
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss_value = self.loss.forward(y_pred, y)
            g_output = self.loss.backward(y_pred, y)
            self.backward(g_output)
            self.optimizer.step(self.layers)
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss_value:.4f}')

    def predict(self, X):
        """
        Parameters:
        - X (ndarray): Input data

        Returns:
        - ndarray: Predicted output
        """
        return self.forward(X)

    def evaluate(self, X, y):
        """
        Parameters:
        - X (ndarray): Input data
        - y (ndarray): True labels

        Returns:
        - tuple: Loss value and accuracy
        """
        y_pred = self.predict(X)
        loss_value = self.loss.forward(y_pred, y)
        accuracy = np.mean(np.argmax(y_pred, axis=1) == y)
        print(f'Loss: {loss_value:.4f}, Accuracy: {accuracy:.4f}')
        return loss_value, accuracy

model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

loss = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.2)
model.compile(loss, optimizer)

model.train(x_train, y_train, epochs=150)

test_loss, test_accuracy = model.evaluate(X_dev, Y_dev)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')
